# Pickup to Delivery Overall

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from haversine import haversine, Unit
from sklearn.metrics.pairwise import haversine_distances, manhattan_distances
from sklearn.model_selection import train_test_split, cross_val_score

sys.path.insert(0, os.path.expanduser('./'))
import query_runner as qr
import utils
from estimator import BaselineModel_sum, BaselineModel_mean, LinearModel

In [2]:
base_query_path = './queries/'
dwh_config, livedb_config, parameters_config = utils.load_config(config_file='./config.ini')
datalake_connection = qr.create_connection(db='datalake')
#monolith_connection = qr.create_connection(user=livedb_config['monolith_username'], password=livedb_config['monolith_password'], db='livedb')
#dispatching_db_connection = qr.create_connection(user=livedb_config['dispatching_db_username'], password=livedb_config['dispatching_db_password'], db='dispatchingdb')

INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.


In [3]:
start_date = parameters_config['start_date']
end_date = parameters_config['end_date']
country_code = parameters_config['country_code']
cities = parameters_config['cities']

print(f'Start date: {start_date} | End date: {end_date} | Countries: {country_code} | Cities: {cities}')

Start date: 2024-09-30 | End date: 2024-10-20 | Countries: ES | Cities: 'MAD', 'BCN', 'SEV', 'ALC'


In [4]:
parameters = {
    'start_date': start_date,
    'end_date': end_date,
    'country_code': country_code,
    'cities': cities
}

## Load the data

In [ ]:
query_name = '''
SELECT
    olf.country_code                                 AS country_code,
    olf.city_code                                    AS city_code,
    olf.order_id                                     AS order_id,
    olf.courier_id                                   AS courier_id,
    olf.order_created_local_datetime                 AS creation_timestamp,
    olf.order_activated_local_datetime               AS activation_timestamp,
    olf.courier_transport                            AS transport,
    olf.order_picked_up_local_datetime               AS pickup_timestamp,
    olf.order_arrival_to_delivery_local_datetime     AS delivery_entering_timestamp,
    olf.order_delivered_local_datetime               AS delivery_timestamp,
    olf.order_pickup_latitude                        AS pickup_latitude,
    olf.order_pickup_longitude                       AS pickup_longitude,
    olf.order_delivery_latitude                      AS delivery_latitude,
    olf.order_delivery_longitude                     AS delivery_longitude,
    olf.order_time_zone                              AS time_zone,
    olf.p_creation_date
FROM delta.courier_routing_courier_ml_features_odp.order_level_features AS olf
WHERE order_final_status = 'DeliveredStatus'
    AND order_number_of_assignments = 1
    AND order_bundle_index IS NULL
    AND p_creation_date >= DATE '[start_date]' AND p_creation_date < DATE '[end_date]'
    AND country_code IN ('[country_code]')
    AND city_code IN ([cities])
'''

query = qr.Query(base_query_path, query_name, datalake_connection, parameters_dict=parameters, query_from_file = False)

df = query.run()
df = df.fillna(value=np.nan)

data = df.copy()
data.head()

## Clean the dataset

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
# Check for missing values
data.isnull().sum()

In [ ]:
# print the number of null rows
data.isnull().sum().sum()

In [ ]:
# Remove rows with null values: we have many rows, so we can afford to remove them
data.dropna(inplace=True)

In [ ]:
# Check for missing values
data.isnull().sum()

In [ ]:
# Check for duplicates
data.duplicated().sum()

## Compute new features

In [ ]:
# Convert the creation time to datetime
data['creation_timestamp'] = pd.to_datetime(data['creation_timestamp'])
data['activation_timestamp'] = pd.to_datetime(data['activation_timestamp'])
data['pickup_timestamp'] = pd.to_datetime(data['pickup_timestamp'])
data['delivery_timestamp'] = pd.to_datetime(data['delivery_timestamp'])
data['delivery_entering_timestamp'] = pd.to_datetime(data['delivery_entering_timestamp'])

# Compute the delivery date and the delivery time
data['creation_date'] = data['creation_timestamp'].dt.date
data['creation_time'] = data['creation_timestamp'].dt.time
data['creation_hour'] = data['creation_timestamp'].dt.hour

To determine whether a coordinate is in degrees or radians, you can consider the typical ranges and values for latitude and longitude:
1. **Degrees:**
   - Latitude ranges from -90 to 90 degrees.
   - Longitude ranges from -180 to 180 degrees.
   - Values are typically whole numbers or decimals within these ranges.
2. **Radians:**
   - Latitude and longitude in radians will range from approximately -π/2 to π/2 for latitude and -π to π for longitude.
   - Values are typically small decimals (e.g., 0.5, 1.0, etc.).

Given our dataset, as the values in the columns `pickup_latitude`, `pickup_longitude`, `delivery_latitude`, `delivery_longitude` fall within the typical range for degrees, it is safe to assume that these coordinates are in degrees.

There is a difference in how the `haversine` library, the `sklearn`'s `haversine_distances`, and the `sklearn`'s `manhattan_distances` function compute and return the distances. Let's break down the differences and how to resolve them:
1. **Haversine Library:**
   - The `haversine` library directly computes the distance between two points and returns a single scalar value.
2. **Sklearn's `haversine_distances`:**
   - The `haversine_distances` function from `sklearn` returns a distance matrix. When you input two points, it returns a 1x1 matrix (a nested list) containing the distance. This is why you would see the result in squared parentheses like `[[]]`. We extract the single value using `[0][0]`.
   - To use these coordinates with sklearn's `haversine_distances` function, you need to convert them to radians using `np.radians`.
   - Additionally, the `haversine_distances` function returns the distance in radians, not in meters. To convert this to meters, you need to multiply by the Earth's radius (approximately 6371000 meters).
3. **Sklearn's `manhattan_distances`:**
   - The `manhattan_distances` function computes the Manhattan distance between two points and returns a distance matrix. We extract the single value from the 1x1 matrix using `[0][0]`.
   - Additionally, the `manhattan_distances` function from sklearn computes the distance based on the Cartesian coordinates provided. Since latitude and longitude are angular measurements, the result will not be in meters but in degrees. To convert the Manhattan distance from degrees to meters, you need to account for the Earth's curvature. 
      - The conversion factor for latitude is approximately 111,320 meters per degree.
      - The conversion factor for longitude varies based on the latitude. At the equator, it's approximately 111,320 meters per degree, but it decreases as you move towards the poles.
      - Convert the latitude and longitude differences to meters.
      - Sum the absolute differences to get the Manhattan distance in meters.

In [ ]:
# Convert degrees to radians
data['pickup_latitude_rad'] = np.radians(data['pickup_latitude'])
data['pickup_longitude_rad'] = np.radians(data['pickup_longitude'])
data['delivery_latitude_rad'] = np.radians(data['delivery_latitude'])
data['delivery_longitude_rad'] = np.radians(data['delivery_longitude'])

# Earth's radius in meters
earth_radius_m = 6371.0088 * 1000  # average earth radius - https://en.wikipedia.org/wiki/Earth_radius#Mean_radius

# Conversion factors
meters_per_degree_lat = 111320  # Approximate meters per degree of latitude

def manhattan_distance_in_meters(row):
    # Convert latitude and longitude differences to meters
    lat_diff_m = abs(row['pickup_latitude'] - row['delivery_latitude']) * meters_per_degree_lat
    # Convert longitude difference to meters, considering the latitude
    lon_diff_m = abs(row['pickup_longitude'] - row['delivery_longitude']) * meters_per_degree_lat * np.cos(np.radians((row['pickup_latitude'] + row['delivery_latitude']) / 2))
    # Sum the absolute differences to get the Manhattan distance in meters
    return lat_diff_m + lon_diff_m

In [ ]:
data['pd_distance_haversine_m'] = data.apply(
    lambda x: haversine(
        (x['pickup_latitude'], x['pickup_longitude']),
        (x['delivery_latitude'], x['delivery_longitude']),
        unit=Unit.METERS
    ), axis=1
)
data['pd_distance_haversine_m_sk'] = data.apply(
    lambda x: haversine_distances(
        np.array([[x['pickup_latitude_rad'], x['pickup_longitude_rad']]]),
        np.array([[x['delivery_latitude_rad'], x['delivery_longitude_rad']]])
    )[0][0] * earth_radius_m, axis=1
)
data['pd_distance_manhattan_m'] = data.apply(manhattan_distance_in_meters, axis=1)
data.head()

In [ ]:
# print the number of null rows
data.isnull().sum().sum()

## Save the dataset

It's better to use the parquet format, as it is more efficient and faster to read and write. Besides, it is a columnar format, which is more suitable for analytical queries. We can also partition the data by creation date and city, which will help to speed up the queries and allows to analyze different timeframes and different cities if needed.

In [ ]:
# parquet appends the data in the files, it doesn't overwrite them, so we need to manually remove the folder with its content to avoid duplicated data
shutil.rmtree("data/parquet/")
os.makedirs("data/parquet/")

In [ ]:
data.to_parquet("data/parquet/dataframe.parquet", index=False, partition_cols=['creation_date', 'city_code'])

## Load the dataset

In [5]:
data = pd.read_parquet("data/parquet/dataframe.parquet")

## Exploratory Data Analysis (EDA)

In [ ]:
# Histogram of the # of data per day / hour
plt.figure(figsize=(15, 8))
plt.hist(data['creation_timestamp'], bins = 1000)
plt.title('Histogram of the # of data per day / hour')
plt.xlabel('Day / Hour')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(data['creation_date'], bins = 14)
plt.title('Histogram of the # of data per day')
plt.xlabel('Day')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(data['creation_hour'])
plt.title('Histogram of the # of data per hour')
plt.xlabel('Hour')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Check the distribution of the transport types
data['transport'].value_counts()

In [ ]:
# Check the distribution of the distances
plt.figure(figsize=(15, 8))
plt.hist(data['pd_distance_haversine_m'], bins = 1000)
plt.xlim(0, 10000)
plt.title('Histogram of the distances')
plt.xlabel('Distance (m)')
plt.ylabel('Frequency')
plt.show()

## Hyperparameters

In [6]:
test_set_perc = 0.1
days_for_test = 7
k_cv = 5

## Database split

In [7]:
X = data
y = data['delivery_entering_timestamp'] - data['pickup_timestamp']
y = pd.Series(y, name='pickup_to_delivery')
y

0        0 days 00:12:22.108000
1        0 days 00:04:47.417000
2        0 days 00:08:15.392000
3        0 days 00:02:55.062000
4        0 days 00:06:16.423000
                  ...          
972466   0 days 00:02:35.306000
972467   0 days 00:06:11.268000
972468   0 days 00:12:22.575000
972469   0 days 00:13:06.619000
972470   0 days 00:03:14.149000
Name: pickup_to_delivery, Length: 972471, dtype: timedelta64[ns]

As we are dealing with a time-series dataset (orders are placed at different times), we will split the data based on the creation timestamp, leaving out the last 10% of the data for testing. This will help to understand the performance of the model on unseen data, as in reality we will have to test the model on data created on day+1 with respect to our training data.

In [8]:
X.sort_values('creation_timestamp', inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_perc, random_state=0)

In [9]:
X_train

,country_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_entering_timestamp,delivery_timestamp,pickup_latitude,...,creation_hour,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,creation_date,city_code
829953,ES,100939319797,107746007,2024-10-17 23:34:35+00:00,2024-10-17 23:34:36+00:00,BICYCLE,2024-10-17 23:44:41.077000+00:00,2024-10-17 23:54:16+00:00,2024-10-17 23:55:49+00:00,41.390423,...,23,0.722399,0.037836,0.722325,0.037685,860.626737,860.626737,1192.918831,2024-10-17,BCN
669859,ES,100928878014,159994081,2024-10-13 10:15:57+00:00,2024-10-13 10:15:58+00:00,MOTORBIKE,2024-10-13 10:54:50.678000+00:00,2024-10-13 11:12:48+00:00,2024-10-13 11:15:25+00:00,40.446358,...,10,0.705922,-0.062614,0.705962,-0.062928,1543.613921,1543.613921,1778.636018,2024-10-13,MAD
140760,ES,100907146806,56424684,2024-10-03 19:11:31+00:00,2024-10-03 19:11:31+00:00,BICYCLE,2024-10-03 19:17:37.563000+00:00,2024-10-03 19:21:24+00:00,2024-10-03 19:24:36+00:00,41.404724,...,19,0.722649,0.038259,0.722660,0.038404,695.706194,695.706194,764.678030,2024-10-03,BCN
452412,ES,100919437062,84525678,2024-10-09 00:44:00+00:00,2024-10-09 00:44:00+00:00,BICYCLE,2024-10-09 00:52:46.420000+00:00,2024-10-09 01:07:02+00:00,2024-10-09 01:07:53+00:00,41.392460,...,0,0.722435,0.038459,0.722945,0.038384,3267.345470,3267.345470,3607.004626,2024-10-09,BCN
321294,ES,100913100187,9128920,2024-10-06 10:39:27+00:00,2024-10-06 10:39:29+00:00,CAR,2024-10-06 10:57:47.651000+00:00,2024-10-06 11:00:02+00:00,2024-10-06 11:01:20+00:00,40.418365,...,10,0.705434,-0.064176,0.705491,-0.064168,370.807625,370.807625,409.366432,2024-10-06,MAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172666,ES,100906861608,108265471,2024-10-03 17:21:28+00:00,2024-10-03 17:21:29+00:00,MOTORBIKE,2024-10-03 17:30:44.205000+00:00,2024-10-03 17:35:51+00:00,2024-10-03 17:38:36+00:00,40.432785,...,17,0.705685,-0.064283,0.705831,-0.064182,1046.507571,1046.507571,1415.105660,2024-10-03,MAD
918618,ES,100944115229,169107249,2024-10-19 21:53:26+00:00,2024-10-19 21:53:28+00:00,BICYCLE,2024-10-19 22:01:57.618000+00:00,2024-10-19 22:04:29+00:00,2024-10-19 22:06:33+00:00,41.393707,...,21,0.722456,0.037507,0.722471,0.037606,481.520294,481.520294,565.340153,2024-10-19,BCN
111084,ES,100905287793,177356926,2024-10-02 20:24:46+00:00,2024-10-02 20:28:42+00:00,MOTORBIKE,2024-10-02 20:44:55.616000+00:00,2024-10-02 21:13:39+00:00,2024-10-02 21:20:03+00:00,40.338730,...,20,0.704044,-0.065159,0.703370,-0.065534,4660.618245,4660.618245,6117.422493,2024-10-02,MAD
437374,ES,100919914573,25564120,2024-10-09 13:12:58+00:00,2024-10-09 13:12:59+00:00,MOTORBIKE,2024-10-09 13:32:35.416000+00:00,2024-10-09 13:43:15+00:00,2024-10-09 13:44:38+00:00,41.435143,...,13,0.723180,0.038061,0.722814,0.037869,2500.692483,2500.692483,3247.075034,2024-10-09,BCN


In [10]:
y_train

858031   0 days 00:12:10.939000
626108   0 days 00:07:52.667000
155855   0 days 00:00:03.892000
427956   0 days 00:03:16.071000
294097   0 days 00:08:52.655000
                  ...          
152315   0 days 00:14:41.211000
963395   0 days 00:04:11.996000
117952   0 days 00:05:22.682000
435829   0 days 00:17:50.353000
305711   0 days 00:05:46.393000
Name: pickup_to_delivery, Length: 875223, dtype: timedelta64[ns]

In [11]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(875223, 26) (875223,)
(97248, 26) (97248,)


In [12]:
# In case we want to test different hyperparameters, we will use cross-validation
#scores = cross_val_score(<estimator>, X, y, cv=k_cv)

### Database split using directly the creation date

As we have partitioned the data by city and creation date, we can use this information to split the data. This will help to avoid data leakage, as we will not have data from the future in the training set.
This is much better than just sorting the data by the creation timestamp and taking 10% of the dataset as test set, as we did before.

In [13]:
# We take the last week of the dataset to test the model
begin_test_date = pd.to_datetime(end_date) - pd.Timedelta(days=days_for_test-1)
begin_test_date = begin_test_date.strftime("%Y-%m-%d")
print(f'Start date: {start_date} | Begin test date: {begin_test_date} | End date: {end_date}')

Start date: 2024-09-30 | Begin test date: 2024-10-14 | End date: 2024-10-20


In [14]:
X_train = pd.read_parquet("data/parquet/dataframe.parquet/", filters=[('creation_date', '<', begin_test_date)])
X_train.head()

,country_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_entering_timestamp,delivery_timestamp,pickup_latitude,...,creation_hour,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,creation_date,city_code
0,ES,100901312508,24750184,2024-09-30 20:48:58+00:00,2024-09-30 20:48:59+00:00,CAR,2024-09-30 21:05:11.892000+00:00,2024-09-30 21:17:34+00:00,2024-09-30 21:20:22+00:00,38.391617,...,20,0.670060,-0.008959,0.670057,-0.009104,724.460401,724.460401,742.866745,2024-09-30,ALC
1,ES,100901502888,114878043,2024-09-30 22:02:24+00:00,2024-09-30 22:02:25+00:00,MOTORBIKE,2024-09-30 22:18:10.583000+00:00,2024-09-30 22:22:58+00:00,2024-09-30 22:23:44+00:00,38.338440,...,22,0.669132,-0.008492,0.669226,-0.008693,1166.853169,1166.853169,1601.094467,2024-09-30,ALC
2,ES,100901552273,169065712,2024-09-30 22:29:26+00:00,2024-09-30 22:29:26+00:00,CAR,2024-09-30 22:42:27.608000+00:00,2024-09-30 22:50:43+00:00,2024-09-30 22:56:36+00:00,38.366512,...,22,0.669622,-0.008200,0.669375,-0.008726,3065.108653,3065.108653,4208.852129,2024-09-30,ALC
3,ES,100901587119,98812119,2024-09-30 22:53:13+00:00,2024-09-30 22:53:13+00:00,CAR,2024-09-30 23:13:58.938000+00:00,2024-09-30 23:16:54+00:00,2024-09-30 23:21:06+00:00,38.359184,...,22,0.669494,-0.008437,0.669460,-0.008370,400.794655,400.794655,555.236086,2024-09-30,ALC
4,ES,100901593554,170866593,2024-09-30 22:58:15+00:00,2024-09-30 22:58:17+00:00,MOTORBIKE,2024-09-30 23:08:24.577000+00:00,2024-09-30 23:14:41+00:00,2024-09-30 23:16:10+00:00,38.345978,...,22,0.669264,-0.008514,0.669439,-0.008415,1225.327574,1225.327574,1618.610270,2024-09-30,ALC


In [15]:
# Check that there are no nulls deriving from a wrong writing of parquet files (appending instead of overwriting)
X_train.isnull().sum().sum()

np.int64(0)

In [16]:
y_train = X_train['delivery_entering_timestamp'] - X_train['pickup_timestamp']
y_train = pd.Series(y_train, name='pickup_to_delivery')
y_train

0        0 days 00:12:22.108000
1        0 days 00:04:47.417000
2        0 days 00:08:15.392000
3        0 days 00:02:55.062000
4        0 days 00:06:16.423000
                  ...          
678013   0 days 00:04:27.832000
678014   0 days 00:05:29.981000
678015   0 days 00:12:19.778000
678016   0 days 00:08:33.237000
678017   0 days 00:09:04.232000
Name: pickup_to_delivery, Length: 678018, dtype: timedelta64[ns]

In [17]:
y_train.isnull().sum().sum()

np.int64(0)

In [18]:
X_test = pd.read_parquet("data/parquet/dataframe.parquet", filters=[('creation_date', '>=', begin_test_date)])
X_test.head()

,country_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_timestamp,delivery_entering_timestamp,delivery_timestamp,pickup_latitude,...,creation_hour,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,creation_date,city_code
0,ES,100931299686,172175095,2024-10-14 00:09:32+00:00,2024-10-14 00:09:34+00:00,MOTORBIKE,2024-10-14 00:28:54.317000+00:00,2024-10-14 00:34:47+00:00,2024-10-14 00:36:48+00:00,38.382520,...,0,0.669901,-0.007882,0.669500,-0.008304,3310.852444,3310.852444,4665.700375,2024-10-14,ALC
1,ES,100931323115,170201413,2024-10-14 00:50:44+00:00,2024-10-14 10:12:43+00:00,CAR,2024-10-14 10:36:43.867000+00:00,2024-10-14 10:48:41+00:00,2024-10-14 10:53:34+00:00,38.348476,...,0,0.669307,-0.008489,0.669187,-0.008794,1705.517814,1705.517814,2292.240241,2024-10-14,ALC
2,ES,100931293587,2320936,2024-10-14 00:00:33+00:00,2024-10-14 00:00:34+00:00,CAR,2024-10-14 00:11:05.783000+00:00,2024-10-14 00:16:49+00:00,2024-10-14 00:17:38+00:00,38.425022,...,0,0.670643,-0.007077,0.670889,-0.006727,2345.425073,2345.425073,3315.503069,2024-10-14,ALC
3,ES,100931297084,48156385,2024-10-14 00:05:52+00:00,2024-10-14 00:05:53+00:00,MOTORBIKE,2024-10-14 00:25:18.815000+00:00,2024-10-14 00:33:40+00:00,2024-10-14 00:40:31+00:00,38.348465,...,0,0.669307,-0.008617,0.669430,-0.008205,2201.177479,2201.177479,2843.624975,2024-10-14,ALC
4,ES,100931307238,177355044,2024-10-14 00:20:58+00:00,2024-10-14 00:20:59+00:00,MOTORBIKE,2024-10-14 00:39:33.601000+00:00,2024-10-14 00:51:56+00:00,2024-10-14 00:57:26+00:00,38.338870,...,0,0.669140,-0.009017,0.669468,-0.008548,3138.707723,3138.707723,4436.739653,2024-10-14,ALC


In [19]:
X_test.isnull().sum().sum()

np.int64(0)

To compute the MAE, we need to do a power to 2, but if I use the type `np.timedelta64(1, "ns")` for `y_test` I get the following error:
`TypeError: cannot perform __pow__ with this index type: TimedeltaArray`
Therefore we will use the type `np.float64` for `y_test`.

In [20]:
y_test = (X_test['delivery_entering_timestamp'] - X_test['pickup_timestamp']).dt.total_seconds()
y_test = pd.Series(y_test, dtype=np.float64, name='pickup_to_delivery')
y_test

0         352.683
1         717.133
2         343.217
3         501.185
4         742.399
           ...   
294448    155.306
294449    371.268
294450    742.575
294451    786.619
294452    194.149
Name: pickup_to_delivery, Length: 294453, dtype: float64

In [21]:
y_test.isnull().sum().sum()

np.int64(0)

In [22]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(678018, 26) (678018,)
(294453, 26) (294453,)


## Baseline Models

### BaselineModel_sum

In [ ]:
model_bl_sum = BaselineModel_sum()
model_bl_sum.fit(X_train, y_train)

In [ ]:
X_test_expanded = X_test.copy()
X_test_expanded['y_test_predicted'] = model_bl_sum.predict(X_test)
X_test_expanded['y_test'] = (X_test_expanded['delivery_entering_timestamp'] - X_test_expanded['pickup_timestamp']).dt.total_seconds()
X_test_expanded['diff'] = X_test_expanded['y_test_predicted'] - X_test_expanded['y_test']
X_test_expanded

In [ ]:
model_bl_sum.predict(X_test.iloc[0])

### BaselineModel_mean

In [ ]:
model_bl_mean = BaselineModel_mean()
model_bl_mean.fit(X_train, y_train)

In [ ]:
X_test_expanded2 = X_test.copy()
X_test_expanded2['y_test_predicted'] = model_bl_mean.predict(X_test)
X_test_expanded2['y_test'] = (X_test_expanded2['delivery_entering_timestamp'] - X_test_expanded2['pickup_timestamp']).dt.total_seconds()
X_test_expanded2['diff'] = X_test_expanded2['y_test_predicted'] - X_test_expanded2['y_test']
X_test_expanded2

In [ ]:
model_bl_mean.predict(X_test.iloc[0])

## Evaluation pipeline

In [ ]:
model_bl_sum.evaluate(X_test, y_test)

In [ ]:
X_test.shape

In [ ]:
model_bl_mean.evaluate(X_test, y_test)

## Linear Model

In [ ]:
model_linear = LinearModel()
model_linear.fit(X_train, y_train)

In [ ]:
X_test_expanded3 = X_test.copy()
X_test_expanded3['y_test_predicted'] = model_linear.predict(X_test)
X_test_expanded3['y_test'] = (X_test_expanded3['delivery_entering_timestamp'] - X_test_expanded3['pickup_timestamp']).dt.total_seconds()
X_test_expanded3['diff'] = X_test_expanded['y_test_predicted'] - X_test_expanded3['y_test']
X_test_expanded3

In [ ]:
model_linear.predict(X_test.iloc[0])

In [ ]:
model_linear.evaluate(X_test, y_test)

Compare performance of the model with and without standardization

In [ ]:
model_linear = LinearModel(standardize=True)
model_linear.fit(X_train, y_train)